In [1]:
import re
import random
from ast import literal_eval
from collections import Counter
import fasttext
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
tqdm.pandas()

/home/abdullah/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
documents = pd.read_csv("final_before_split/immap_dfs_documents.csv")
sentences = pd.read_csv("final_before_split/immap_dfs_sentences.csv")

In [4]:
sentences["sector_ids"] = sentences["sector_ids"].progress_apply(literal_eval)

In [5]:
sec_ids = set()
for sec_ids_list in sentences["sector_ids"]:
    sec_ids.update(sec_ids_list)
sec_ids

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [6]:
sentences["sec_0"] = 0
sentences["sec_1"] = 0
sentences["sec_2"] = 0
sentences["sec_3"] = 0
sentences["sec_4"] = 0
sentences["sec_6"] = 0
sentences["sec_7"] = 0
sentences["sec_8"] = 0
sentences["sec_9"] = 0
sentences["sec_10"] = 0

In [7]:
for row in tqdm(sentences.iterrows(), total=sentences.shape[0]):
    sec_ids = row[1]["sector_ids"]
    idx = row[0]
    if 0 in sec_ids:
        sentences.loc[idx, "sec_0"] = 1
    if 1 in sec_ids:
        sentences.loc[idx, "sec_1"] = 1
    if 2 in sec_ids:
        sentences.loc[idx, "sec_2"] = 1
    if 3 in sec_ids:
        sentences.loc[idx, "sec_3"] = 1
    if 4 in sec_ids:
        sentences.loc[idx, "sec_4"] = 1
    if 5 in sec_ids:
        sentences.loc[idx, "sec_5"] = 1
    if 6 in sec_ids:
        sentences.loc[idx, "sec_6"] = 1
    if 7 in sec_ids:
        sentences.loc[idx, "sec_7"] = 1
    if 8 in sec_ids:
        sentences.loc[idx, "sec_8"] = 1
    if 9 in sec_ids:
        sentences.loc[idx, "sec_9"] = 1
    if 10 in sec_ids:
        sentences.loc[idx, "sec_10"] = 1     

In [11]:
sentences["lang_code"].value_counts()

en    236961
es     59590
fr     59164
Name: lang_code, dtype: int64

In [8]:
sentences_en = sentences[sentences["lang_code"].eq("en")]
sentences_fr = sentences[sentences["lang_code"].eq("fr")]
sentences_es = sentences[sentences["lang_code"].eq("es")]

In [9]:
path_to_pretrained_model = 'lid.176.bin'
fmodel = fasttext.load_model(path_to_pretrained_model)
def detect_lang(doc):
    if isinstance(doc, str):
        doc = re.sub("\s+", " ", doc)
        return fmodel.predict([doc])[0][0][0][len("__label__"):]
    return np.nan

In [10]:
documents["lang_ft"] = documents["doc_text"].progress_apply(detect_lang)

In [12]:
documents.columns

Index(['project_name', 'country_code', 'doc_id', 'doc_text', 'doc_url',
       'lang_ft', 'lang'],
      dtype='object')

In [13]:
documents.country_code.value_counts()

BGD    1160
SYR    1125
NGA     719
BFA     676
COL     652
COD     592
Name: country_code, dtype: int64

In [14]:
documents["lang"] = documents["lang_ft"].copy()
for row in tqdm(documents.iterrows(), total=len(documents)):
    idx = row[0]
    if row[1].lang_ft not in ["es", "en", "fr"]:
        if row[1].country_code == "COL":
            documents.loc[idx, "lang"] = "es"
        elif row[1].country_code in ["SYR", "BGD", "NGA"]:
            documents.loc[idx, "lang"] = "en"
        else:
            documents.loc[idx, "lang"] = "fr"

In [17]:
documents_en = documents[documents["lang"].eq("en")]
documents_fr = documents[documents["lang"].eq("fr")]
documents_es = documents[documents["lang"].eq("es")]

In [19]:
documents.drop('lang_ft', axis=1, inplace=True)

In [20]:
len(documents), len(documents_en), len(documents_fr), len(documents_es)

(4924, 3313, 1016, 595)

In [21]:
def split(docs, train_perc=.8, val_perc=.1, random_state=2021):
    """split a dataframe into train, val, test"""
    train, val, test = np.split(docs.sample(frac=1, random_state=random_state), [int(train_perc*len(docs)), int((train_perc+val_perc)*len(docs))])
    return train, val, test

In [22]:
train_docs_en, val_docs_en, test_docs_en = split(documents_en)
train_docs_fr, val_docs_fr, test_docs_fr = split(documents_fr)
train_docs_es, val_docs_es, test_docs_es = split(documents_es)
## write en
train_docs_en.to_csv("final_data_random_split/en/documents_en_train.csv", index=None)
val_docs_en.to_csv("final_data_random_split/en/documents_en_val.csv", index=None)
test_docs_en.to_csv("final_data_random_split/en/documents_en_test.csv", index=None)
## write fr
train_docs_fr.to_csv("final_data_random_split/fr/documents_fr_train.csv", index=None)
val_docs_fr.to_csv("final_data_random_split/fr/documents_fr_val.csv", index=None)
test_docs_fr.to_csv("final_data_random_split/fr/documents_fr_test.csv", index=None)
## write es
train_docs_es.to_csv("final_data_random_split/es/documents_es_train.csv", index=None)
val_docs_es.to_csv("final_data_random_split/es/documents_es_val.csv", index=None)
test_docs_es.to_csv("final_data_random_split/es/documents_es_test.csv", index=None)

In [23]:
## en
train_sentences_en = sentences[sentences["doc_id"].isin(train_docs_en["doc_id"])]
val_sentences_en = sentences[sentences["doc_id"].isin(val_docs_en["doc_id"])]
test_sentences_en = sentences[sentences["doc_id"].isin(test_docs_en["doc_id"])]
## write
train_sentences_en.to_csv("final_data_random_split/en/sentences_en_train.csv", index=None)
val_sentences_en.to_csv("final_data_random_split/en/sentences_en_val.csv", index=None)
test_sentences_en.to_csv("final_data_random_split/en/sentences_en_test.csv", index=None)
test_sentences_en.to_csv("final_data_random_split/en/sentences_en_test_without_ground_truth.csv", index=None, columns=['doc_id', 'sentence_id', 'sentence_text'])

In [24]:
## fr
train_sentences_fr = sentences[sentences["doc_id"].isin(train_docs_fr["doc_id"])]
val_sentences_fr = sentences[sentences["doc_id"].isin(val_docs_fr["doc_id"])]
test_sentences_fr = sentences[sentences["doc_id"].isin(test_docs_fr["doc_id"])]
## write
train_sentences_fr.to_csv("final_data_random_split/fr/sentences_fr_train.csv", index=None)
val_sentences_fr.to_csv("final_data_random_split/fr/sentences_fr_val.csv", index=None)
test_sentences_fr.to_csv("final_data_random_split/fr/sentences_fr_test.csv", index=None)
test_sentences_fr.to_csv("final_data_random_split/fr/sentences_fr_test_without_ground_truth.csv", index=None, columns=['doc_id', 'sentence_id', 'sentence_text'])

In [25]:
## es
train_sentences_es = sentences[sentences["doc_id"].isin(train_docs_es["doc_id"])]
val_sentences_es = sentences[sentences["doc_id"].isin(val_docs_es["doc_id"])]
test_sentences_es = sentences[sentences["doc_id"].isin(test_docs_es["doc_id"])]
## write
train_sentences_es.to_csv("final_data_random_split/es/sentences_es_train.csv", index=None)
val_sentences_es.to_csv("final_data_random_split/es/sentences_es_val.csv", index=None)
test_sentences_es.to_csv("final_data_random_split/es/sentences_es_test.csv", index=None)
test_sentences_es.to_csv("final_data_random_split/es/sentences_es_test_without_ground_truth.csv", index=None, columns=['doc_id', 'sentence_id', 'sentence_text'])

In [26]:
sentences_en.sum(numeric_only=True).astype(np.int)

doc_id         10420874061
sentence_id      295368456
is_relevant          41622
sec_0                  918
sec_1                 4557
sec_2                 2291
sec_3                 3926
sec_4                12597
sec_6                 1393
sec_7                 1151
sec_8                 4233
sec_9                 1944
sec_10                2915
sec_5                 3559
dtype: int64

In [28]:
train_sentences_en.sum(numeric_only=True).astype(np.int)

doc_id         8406656283
sentence_id     271130440
is_relevant         32974
sec_0                 724
sec_1                3414
sec_2                1693
sec_3                3154
sec_4               10278
sec_6                1120
sec_7                 811
sec_8                3295
sec_9                1482
sec_10               2235
sec_5                2888
dtype: int64

In [31]:
test_sentences_en.sum(numeric_only=True).astype(np.int)/sentences_en.sum(numeric_only=True).astype(np.int)

doc_id         0.097745
sentence_id    0.025652
is_relevant    0.096632
sec_0          0.068627
sec_1          0.093483
sec_2          0.098647
sec_3          0.068518
sec_4          0.092324
sec_6          0.093324
sec_7          0.084275
sec_8          0.086464
sec_9          0.078704
sec_10         0.093310
sec_5          0.073897
dtype: float64

In [32]:
val_sentences_en.sum(numeric_only=True).astype(np.int)/sentences_en.sum(numeric_only=True).astype(np.int)

doc_id         0.091296
sentence_id    0.086326
is_relevant    0.110639
sec_0          0.140523
sec_1          0.156682
sec_2          0.161938
sec_3          0.127101
sec_4          0.091688
sec_6          0.102656
sec_7          0.208514
sec_8          0.134420
sec_9          0.158436
sec_10         0.139623
sec_5          0.114077
dtype: float64

In [33]:
train_sentences_en.sum(numeric_only=True).astype(np.int)/sentences_en.sum(numeric_only=True).astype(np.int)

doc_id         0.806713
sentence_id    0.917940
is_relevant    0.792225
sec_0          0.788671
sec_1          0.749177
sec_2          0.738979
sec_3          0.803362
sec_4          0.815909
sec_6          0.804020
sec_7          0.704605
sec_8          0.778408
sec_9          0.762346
sec_10         0.766724
sec_5          0.811464
dtype: float64

In [30]:
val_sentences_en.sum(numeric_only=True).astype(np.int)

doc_id         951386867
sentence_id     25497994
is_relevant         4605
sec_0                129
sec_1                714
sec_2                371
sec_3                499
sec_4               1155
sec_6                143
sec_7                240
sec_8                569
sec_9                308
sec_10               407
sec_5                406
dtype: int64